EXTRACCION, TRANSFORMACIÓN Y CARGA ETL

In [1]:

import re
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns



In [6]:
# Se accede a los datasets por dataframes
hechos = pd.read_excel("../datasets/hechos.xlsx")
victimas = pd.read_excel("../datasets/victimas.xlsx")
lesion = pd.read_excel('../datasets/lesiones.xlsx')


El objetivo es limpiar los datos de los dos datasets Hechos y Victimas para  luego unirlos para trabajar un solo dataset en general.

In [7]:
hechos.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO


In [8]:
victimas.head(3)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00


In [9]:
lesion.sample(5)

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
1059,LC-2019-0095461,1,2019,2,13,2019-02-13 00:00:00,23:33:00,23,SD,14,...,-34.585134,CICLISTA,SD,CICLISTA-SD,SD,SD,SD,SD,x,SD
12260,LC-2020-0151195,5,2020,3,9,2020-03-09 00:00:00,11:00:00 a.m.,11,"COSQUIN y RODO, JOSE E.",9,...,-34.660886529208,TRANSPORTE PUBLICO,AUTO,TRANSPORTE PUBLICO-AUTO,0,x,x,0,0,SD
15667,LC-2020-0658216,1,2020,12,14,2020-12-14 00:00:00,07:13:00 p.m.,19,"ALBERDI, JUAN BAUTISTA AV. 2450",7,...,-34.631632128096,SD,SD,SD-SD,0,0,0,0,0,SD
11694,LC-2020-0094232,1,2020,2,13,2020-02-13 00:00:00,12:15:00 a.m.,0,CHACABUCO y BELGRANO AV.,1,...,-34.612751977733,TAXI,TAXI,TAXI-TAXI,0,0,0,0,0,SD
16233,LC-2021-0138317,1,2021,3,3,2021-03-03 00:00:00,12:30:00,12,SUIPACHA y TUCUMAN,1,...,-34.601279574888,CICLISTA,MOTO,CICLISTA-MOTO,x,0,0,0,x,SD


In [10]:
lesion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     23785 non-null  object 
 1   n_victimas             23785 non-null  int64  
 2   aaaa                   23785 non-null  int64  
 3   mm                     23785 non-null  int64  
 4   dd                     23785 non-null  int64  
 5   fecha                  23785 non-null  object 
 6   hora                   23785 non-null  object 
 7   franja_hora            23780 non-null  object 
 8   direccion_normalizada  23732 non-null  object 
 9   comuna                 23616 non-null  object 
 10  tipo_calle             23785 non-null  object 
 11  otra_direccion         23785 non-null  object 
 12  calle                  12867 non-null  object 
 13  altura                 12771 non-null  float64
 14  cruce                  9407 non-null   object 
 15  ge

In [11]:
hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

In [12]:
hechos.describe()

,N_VICTIMAS,FECHA,AAAA,MM,DD,Altura,COMUNA
count,696.000000,696,696.000000,696.000000,696.000000,129.000000,696.000000
mean,1.030172,2018-09-14 05:12:24.827586048,2018.188218,6.692529,15.936782,3336.635659,7.425287
min,1.000000,2016-01-01 00:00:00,2016.000000,1.000000,1.000000,30.000000,0.000000
25%,1.000000,2017-04-02 12:00:00,2017.000000,4.000000,9.000000,1359.000000,4.000000
50%,1.000000,2018-07-12 12:00:00,2018.000000,7.000000,16.000000,2551.000000,8.000000
75%,1.000000,2020-01-04 06:00:00,2020.000000,10.000000,23.000000,4500.000000,11.000000
max,3.000000,2021-12-30 00:00:00,2021.000000,12.000000,31.000000,16080.000000,15.000000
std,0.179393,NaN,1.683754,3.571309,8.639646,3060.641793,4.387050


Se observa que los datasets hechos y victimas contiene varios campos marcados como SIN DATOS (SD), por lo cual para unificar esto se reeemplaza este valor por NAN:

In [13]:
hechos.replace(['SD','sd'], np.nan, inplace=True)


C:\Users\jupai\AppData\Local\Temp\ipykernel_14728\798793323.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  hechos.replace(['SD','sd'], np.nan, inplace=True)


Se analizan la cantidad de campos nulos en  dataset hechos 

In [14]:
hechos.isnull().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       1
HH                         1
LUGAR_DEL_HECHO            1
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    9
ACUSADO                   23
dtype: int64

El dataset de Hechos contiene 696 registros en total. Se observa que la columna ALTURA, contiene 567 registros nulos y esta calumna representa la calle donde se produce el siniestro o accidente. Por lo tanto esta información se torna innecesaria y se puede elimnar del dataset. Tambien se elimina la columna PARTICIPANTES  ya que contiene la misma información que las columnas VICTIMA y ACUSADO. Estas columnas tambien se mantiene los campos nulos para evitar afectación en los análisis posteriores.
La columna CRUCE tiene 171 registros nulos sin embargo se mantiene estos datos para no afectar las otras columnas a combinar infomración. De igual manera se mantiene los registros nulos de la columna CALLE y DIRECCION NORMALIZADA, para no afectar la estadística final.

El dataset victimas contien 717 registros y no contiene registros nulos. sin mebaro contiene unas columnas que se repiten con el dataset Hechos, por lo cual se eliminan.

In [15]:
hechos= hechos.drop(["Altura", 'PARTICIPANTES'], axis=1)

In [16]:
hechos.sample(5)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,VICTIMA,ACUSADO
287,2018-0013,1,2018-01-28,2018,1,28,08:15:00,8.0,Congreso Av. y Galvan,AVENIDA,CONGRESO AV.,GALVAN,CONGRESO AV. y GALVAN,12,Point (97860.62977748 106812.00802724),-58.48661286,-34.56786129,MOTO,AUTO
247,2017-0129,1,2017-10-26,2017,10,26,19:15:15,19.0,AU PERITO MORENO Y AU 25 DE MAYO,AUTOPISTA,AUTOPISTA PERITO MORENO,AUTOPISTA 25 DE MAYO,AUTOPISTA PERITO MORENO y AUTOPISTA 25 DE MAYO,9,Point (99243.41866587 97782.67879316),-58.47155277,-34.64925746,MOTO,PASAJEROS
177,2017-0045,1,2017-04-14,2017,4,14,06:15:00,6.0,NOGOYA 5940,CALLE,NOGOYA,NaN,NOGOYA 5940,10,Point (94388.40460408 100522.78812728),-58.52449285,-34.62454175,MOTO,OBJETO FIJO
109,2016-0139,1,2016-10-28,2016,10,28,23:00:00,23.0,AV PRESIDENTE FIGUEROA ALCORTA Y AV DORREGO,AVENIDA,"FIGUEROA ALCORTA, PRES. AV.",DORREGO AV.,"FIGUEROA ALCORTA, PRES. AV. y DORREGO AV.",14,Point (103965.45958231 107084.94812768),-58.42008769,-34.56539579,MOTO,MOTO
228,2017-0106,1,2017-08-31,2017,8,31,19:50:00,19.0,FRAY JUSTO SANTAMARIA DE ORO Y GUEMES,CALLE,FRAY JUSTO SANTAMARIA DE ORO,GUEMES,FRAY JUSTO SANTAMARIA DE ORO y GUEMES,14,Point (103483.20578848 105425.94020024),-58.42533612,-34.58035258,PEATON,PASAJEROS


Se verifica si hay registros o filas duplicadas:

In [17]:
hechos.duplicated().sum()

0

Se observa que la columna XY (CABA) contiene las coordenas del lugar del accidente o siniestro. Con esta información podemos ubicar en el mapa correspondiente para un mejor analisis. Se procede a usar la siguiente función para extraer los datos y crear dos columnas, asi:

In [18]:

# Esta función toma un argumento de entrada para extraer las coordenadas
# Utiliza una expresión regular (re.findall) para buscar todas las coincidencias de números decimales 
#(que tienen un punto como separador decimal) en el texto. Estas coincidencias se almacenan 
#en la lista coordenadas.
# return float(coordenadas[0]), float(coordenadas[1]): Si se encontraron dos coordenadas,
# se convierten en números de punto flotante (float) y se devuelven como una tupla de dos valores.
# Luego, se aplica esta función a la columna 'XY (CABA)' de df_siniestros utilizando el método .map().
# Esto se hace para cada fila de la columna, y los resultados se almacenan en dos nuevas columnas, 'coordenada_x' y 'coordenada_y', utilizando zip(*...).

def extraer_coordenadas(texto):
    # Utilizamos una expresión regular para extraer las coordenadas
    coordenadas = re.findall(r'\d+\.\d+', texto)
    if len(coordenadas) == 2:
        return float(coordenadas[0]), float(coordenadas[1])
    else:
        return None, None

# Aplicar la función a la columna 'XY (CABA)'
hechos['coordenada_x'], hechos['coordenada_y'] = zip(*hechos['XY (CABA)'].map(extraer_coordenadas))

# Filtrar los registros que no tienen coordenadas válidas
hechos = hechos[(hechos['coordenada_x'].notnull()) & (hechos['coordenada_y'].notnull())]

In [19]:
hechos= hechos.drop('XY (CABA)', axis=1)

In [20]:
hechos = hechos.rename(columns={'ID': 'Id'})

In [21]:
hechos.head(3)

,Id,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,pos x,pos y,VICTIMA,ACUSADO,coordenada_x,coordenada_y
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,-58.47533969,-34.68757022,MOTO,AUTO,98896.782384,93532.434378
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,-58.50877521,-34.66977709,AUTO,PASAJEROS,95832.055711,95505.416420
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7.0,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034,1,-58.39040293,-34.63189362,MOTO,AUTO,106684.290900,99706.576878


DATASET VICTIMAS

In [23]:
victimas = pd.read_excel("../datasets/victimas.xlsx")

In [24]:
#Cambio la columna a tipo datetime
victimas["FECHA_FALLECIMIENTO"] = pd.to_datetime(victimas["FECHA_FALLECIMIENTO"], format='%Y-%m-%d', errors='coerce')

In [25]:
#Una vez que se paso el tipo de dato a datetime que es una fecha y hora completa se utiliza este codigo para que solo se quede en formato fecha sin hora
victimas["FECHA_FALLECIMIENTO"] = victimas["FECHA_FALLECIMIENTO"].dt.date

#Luego se reemplazan las fechas en cero con espacios vacios a fin de su manipulación
victimas["FECHA_FALLECIMIENTO"] = victimas ["FECHA_FALLECIMIENTO"].replace(pd.NaT,"")

In [26]:
#Cambio la columna edad a numerico y luego a tipo entero
victimas ["EDAD"] = pd.to_numeric(victimas["EDAD"], errors="coerce")
victimas ["EDAD"] = victimas["EDAD"].astype("Int64")

In [27]:
#Utilizo Numpy para reemplazar por NaN los sin datos o vacios y luego paso a float para que cuando quiera aplicar estadisticas no tenga errores debido a esos datos
victimas["EDAD"] = victimas["EDAD"].replace([" ","SD"], np.nan)

In [28]:
victimas["EDAD"] = victimas["EDAD"].fillna(0.0)

In [29]:
victimas.head(3)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03


In [30]:
victimas.isnull().sum()

ID_hecho               0
FECHA                  0
AAAA                   0
MM                     0
DD                     0
ROL                    0
VICTIMA                0
SEXO                   0
EDAD                   0
FECHA_FALLECIMIENTO    0
dtype: int64

In [31]:
victimas = victimas.rename(columns={'ID_hecho': 'Id'})

Con el objetivo de unir los datasets anteriores, se eliminan las columnas que se encuentran repetidas y con los mismos datos, las cuales son: FECHA, AAAA, MM, DD, VICTIMA.

In [32]:
victimas = victimas.drop(['FECHA', 'AAAA', 'MM', 'DD', 'VICTIMA'], axis=1)

In [33]:
victimas.head()

,Id,ROL,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,CONDUCTOR,MASCULINO,19,2016-01-01
1,2016-0002,CONDUCTOR,MASCULINO,70,2016-01-02
2,2016-0003,CONDUCTOR,MASCULINO,30,2016-01-03
3,2016-0004,CONDUCTOR,MASCULINO,18,
4,2016-0005,CONDUCTOR,MASCULINO,29,2016-02-01


 SE UNIFICAN LOS DOS DATASETS

In [34]:
siniestros= victimas.merge(hechos, on="Id", how="left")

In [35]:
siniestros.head()

,Id,ROL,SEXO,EDAD,FECHA_FALLECIMIENTO,N_VICTIMAS,FECHA,AAAA,MM,DD,...,Calle,Cruce,Dirección Normalizada,COMUNA,pos x,pos y,VICTIMA,ACUSADO,coordenada_x,coordenada_y
0,2016-0001,CONDUCTOR,MASCULINO,19,2016-01-01,1.0,2016-01-01,2016.0,1.0,1.0,...,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8.0,-58.47533969,-34.68757022,MOTO,AUTO,98896.782384,93532.434378
1,2016-0002,CONDUCTOR,MASCULINO,70,2016-01-02,1.0,2016-01-02,2016.0,1.0,2.0,...,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9.0,-58.50877521,-34.66977709,AUTO,PASAJEROS,95832.055711,95505.416420
2,2016-0003,CONDUCTOR,MASCULINO,30,2016-01-03,1.0,2016-01-03,2016.0,1.0,3.0,...,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034,1.0,-58.39040293,-34.63189362,MOTO,AUTO,106684.290900,99706.576878
3,2016-0004,CONDUCTOR,MASCULINO,18,,1.0,2016-01-10,2016.0,1.0,10.0,...,LARRAZABAL AV.,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8.0,-58.46503904,-34.68092974,MOTO,NaN,99840.652248,94269.165344
4,2016-0005,CONDUCTOR,MASCULINO,29,2016-02-01,1.0,2016-01-21,2016.0,1.0,21.0,...,SAN JUAN AV.,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1.0,-58.38718297,-34.62246630,MOTO,PASAJEROS,106980.328279,100752.169158


In [36]:
siniestros["N_VICTIMAS"].unique()

array([ 1.,  2., nan,  3.])

In [37]:
siniestros ["N_VICTIMAS"] = siniestros ["N_VICTIMAS"].fillna(0.0)
siniestros ["N_VICTIMAS"] = siniestros ["N_VICTIMAS"].astype ("Int64")

In [38]:
siniestros["N_VICTIMAS"].unique()

<IntegerArray>
[1, 2, 0, 3]
Length: 4, dtype: Int64

In [39]:
siniestros["AAAA"].unique()

array([2016.,   nan, 2017., 2018., 2019., 2020., 2021.])

In [40]:
siniestros ["AAAA"]= siniestros ["AAAA"].astype ("Int64")
siniestros ["MM"]= siniestros ["MM"].astype ("Int64")
siniestros ["DD"]= siniestros ["DD"].astype ("Int64")

In [41]:
siniestros.head(3)

,Id,ROL,SEXO,EDAD,FECHA_FALLECIMIENTO,N_VICTIMAS,FECHA,AAAA,MM,DD,...,Calle,Cruce,Dirección Normalizada,COMUNA,pos x,pos y,VICTIMA,ACUSADO,coordenada_x,coordenada_y
0,2016-0001,CONDUCTOR,MASCULINO,19,2016-01-01,1,2016-01-01,2016,1,1,...,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8.0,-58.47533969,-34.68757022,MOTO,AUTO,98896.782384,93532.434378
1,2016-0002,CONDUCTOR,MASCULINO,70,2016-01-02,1,2016-01-02,2016,1,2,...,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9.0,-58.50877521,-34.66977709,AUTO,PASAJEROS,95832.055711,95505.416420
2,2016-0003,CONDUCTOR,MASCULINO,30,2016-01-03,1,2016-01-03,2016,1,3,...,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034,1.0,-58.39040293,-34.63189362,MOTO,AUTO,106684.290900,99706.576878


In [42]:
siniestros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Id                     717 non-null    object        
 1   ROL                    717 non-null    object        
 2   SEXO                   717 non-null    object        
 3   EDAD                   717 non-null    Int64         
 4   FECHA_FALLECIMIENTO    717 non-null    object        
 5   N_VICTIMAS             717 non-null    Int64         
 6   FECHA                  702 non-null    datetime64[ns]
 7   AAAA                   702 non-null    Int64         
 8   MM                     702 non-null    Int64         
 9   DD                     702 non-null    Int64         
 10  HORA                   701 non-null    object        
 11  HH                     701 non-null    float64       
 12  LUGAR_DEL_HECHO        702 non-null    object        
 13  TIPO_

In [43]:
nuevos_nombres = ['ID', 'ROL', 'SEXO', 'EDAD', 'FECHA FALLECIMIENTO', 'NUM VICTIMAS', 'FECHA', 'AÑO', 'MES', 'DIA', 'HORA DET', 'HORA', 'LUGAR', 'TIPO CALLE', 'CALLE', 'CRUCE', 'DIRECCION', 'COMUNA', 'POS X', 'POS Y', 'VICTIMA', 'ACUSADO', 'COORD X', 'COORD Y'  ]

In [44]:
siniestros.columns = nuevos_nombres

In [45]:
siniestros ["COMUNA"]= siniestros ["COMUNA"].astype ("Int64")

In [54]:
siniestros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   717 non-null    object        
 1   ROL                  717 non-null    object        
 2   SEXO                 717 non-null    object        
 3   EDAD                 717 non-null    Int64         
 4   FECHA FALLECIMIENTO  717 non-null    object        
 5   NUM VICTIMAS         717 non-null    Int64         
 6   FECHA                702 non-null    datetime64[ns]
 7   AÑO                  702 non-null    Int64         
 8   MES                  702 non-null    Int64         
 9   DIA                  702 non-null    Int64         
 10  HORA DET             701 non-null    object        
 11  HORA                 701 non-null    float64       
 12  LUGAR                702 non-null    object        
 13  TIPO CALLE           702 non-null  

In [55]:
siniestros.isnull().sum()

ID                       0
ROL                      0
SEXO                     0
EDAD                     0
FECHA FALLECIMIENTO      0
NUM VICTIMAS             0
FECHA                   15
AÑO                     15
MES                     15
DIA                     15
HORA DET                16
HORA                    16
LUGAR                   15
TIPO CALLE              15
CALLE                   15
CRUCE                  177
DIRECCION               15
COMUNA                  15
POS X                   15
POS Y                   15
VICTIMA                 22
ACUSADO                 34
COORD X                 15
COORD Y                 15
dtype: int64

In [56]:
siniestros.duplicated().sum()

0

In [51]:
siniestros.to_csv('../datasets/viales.csv',index=False)

In [53]:
siniestros.to_excel('../datasets/vial.xlsx',index=False)